In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import torch

import transformers
from transformers import AutoModel, AutoTokenizer

bert_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(bert_name)
bert = AutoModel.from_pretrained(bert_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [4]:
e = tokenizer.encode("I am hoping for the best", add_special_tokens=True)
print(e)

[101, 1045, 2572, 5327, 2005, 1996, 2190, 102]


In [5]:
q = bert(torch.tensor([e]))
print(q.pooler_output.shape)

torch.Size([1, 768])


In [6]:
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class BertClassifier(nn.Module):
    
    def __init__(self, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        probabilities = self.softmax(logits)
        return logits, probabilities
    

In [7]:
num_classes = 2
model = BertClassifier(num_classes)

In [8]:
e = tokenizer("This is a sequence of classification")

In [9]:
q = model(input_ids=torch.tensor([e['input_ids']]), attention_mask=torch.tensor([e['attention_mask']]))
print(q)

(tensor([[ 0.4823, -0.0793]], grad_fn=<AddmmBackward0>), tensor([[0.6368, 0.3632]], grad_fn=<SoftmaxBackward0>))


In [10]:
from sklearn.datasets import fetch_20newsgroups
from datasets import Dataset, DatasetDict, ClassLabel
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
import torch
import torch.optim as optim

import numpy as np

In [11]:
categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]
remove=("headers", "footers", "quotes")

ng = fetch_20newsgroups(subset="all", categories=categories, 
                              shuffle=True, random_state=42, remove=remove)

X_train, X_temp, y_train, y_temp = train_test_split(ng.data, ng.target, test_size=0.3, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

hf_dataset_dict = {'train': Dataset.from_dict({"text": X_train, "label": y_train}),
              'eval' : Dataset.from_dict({"text": X_eval, "label": y_eval}),
              'test' : Dataset.from_dict({"text": X_test, "label": y_test}),
}
ng_dataset = DatasetDict(hf_dataset_dict)
ng_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2370
    })
    eval: Dataset({
        features: ['text', 'label'],
        num_rows: 508
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 509
    })
})

In [12]:
def tokenize(batch):
  return tokenizer(batch['text'], padding=True, truncation=True)

ng_encoded = ng_dataset.map(tokenize, batched=True, batch_size=None)

ng_encoded.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/2370 [00:00<?, ? examples/s]

Map:   0%|          | 0/508 [00:00<?, ? examples/s]

Map:   0%|          | 0/509 [00:00<?, ? examples/s]

In [13]:
num_classes = len(categories)  # Assuming binary classification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = BertClassifier(num_classes).to(device)

cuda


In [14]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=2e-5)

In [15]:
ng_encoded['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2370
})

In [30]:
from torch.utils.data import DataLoader
dataloader = DataLoader(ng_encoded['train'], batch_size=8, shuffle=True)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, inputs in enumerate(dataloader):
        optimizer.zero_grad()
        outputs, _ = model(inputs['input_ids'].to(device), inputs['attention_mask'].to(device))
        labels = torch.eye(num_classes)[inputs['label']].to(device)        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()        
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(dataloader)}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 24.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 9.06 MiB is free. Process 2581 has 14.74 GiB memory in use. Of the allocated memory 14.50 GiB is allocated by PyTorch, and 110.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
val_loader = DataLoader(ng_encoded['eval'], batch_size=8, shuffle=True)
# Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs in val_loader:
        outputs, _ = model(inputs['input_ids'].to(device), inputs['attention_mask'].to(device))
        _, predicted = torch.max(outputs, 1)
        labels = inputs['label'].to(device)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {correct / total}")

Validation Accuracy: 0.844488188976378
